In [1]:
import numpy
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import LeaveOneOut
import math
import numpy.linalg as LA
from sklearn.cluster import KMeans
import random
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection,metrics
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
random.sample
import pandas as pd
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from collections import Counter

import time
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.model_selection import train_test_split,StratifiedKFold
import matplotlib.pyplot as plt
from utils import *
from tflearn.activations import relu
from optparse import OptionParser
from scipy import interp
from tqdm import trange

/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chujunyi/a

Instructions for updating:
non-resource variables are not supported in the long term


/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [2]:
def sample(all_associations, random_seed):
       
    known_associations = all_associations.loc[all_associations['label'] == 1]
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    random_negative = unknown_associations.sample(n = known_associations.shape[0], random_state = random_seed, axis=0)

    sample_df = known_associations.append(random_negative)
    sample_df.reset_index(drop=True, inplace=True)

    return sample_df

def obtain_data(dtp, isbalance):
    
    if isbalance:
        dtp = sample(dtp, random_seed = 1234)
        
    mirna_ids = list(set(dtp['miRNA']))
    disease_ids = list(set(dtp['disease']))
    random.shuffle(mirna_ids)
    random.shuffle(disease_ids)
    print('# miRNA = {} | Disease = {}'.format(len(mirna_ids), len(disease_ids)))

    mirna_test_num = int(len(mirna_ids) / 5)
    disease_test_num = int(len(disease_ids) / 5)
    print('# Test: miRNA = {} | Disease = {}'.format(mirna_test_num, disease_test_num))    
    
    return dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num

def generate_task_Tp_train_test_idx(dtp):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(dtp.iloc[:, 3:])): #train_index与test_index为下标
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['miRNA', 'disease']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['miRNA', 'disease']]))

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    return train_index_all, test_index_all, train_id_all, test_id_all

def generate_task_Tm_Td_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(np.array(dtp.iloc[train_idx][['miRNA', 'disease']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['miRNA', 'disease']]))
        
    return train_index_all, test_index_all, train_id_all, test_id_all


# 定义任务

In [3]:
parser = OptionParser()
parser.add_option("-d", "--d", default=1024, help="The embedding dimension d")
parser.add_option("-n", "--n", default=1, help="global norm to be clipped")
parser.add_option("-k", "--k", default=512, help="The dimension of project matrices k")
parser.add_option("-t", "--t", default="o", help="Test scenario")
(opts, args) = parser.parse_args(args = [])

def loadtxt(path):
    data = np.loadtxt(path,dtype=str)

    DATA = []
    for i in data:
        DATA.append(i.split(','))

    return np.array(DATA,dtype=float)

def check_symmetric(a, tol=1e-8):
    return np.allclose(a, a.T, atol=tol)

def row_normalize(a_matrix, substract_self_loop):
    if substract_self_loop == True:
        np.fill_diagonal(a_matrix,0)
    a_matrix = a_matrix.astype(float)
    row_sums = a_matrix.sum(axis=1)+1e-12
    new_matrix = a_matrix / row_sums[:, np.newaxis]
    new_matrix[np.isnan(new_matrix) | np.isinf(new_matrix)] = 0.0
    return new_matrix

# load network
network_path = '../data/'
save_path = '../result/'
mirna_name = np.loadtxt(network_path + 'miRNA-name.txt', dtype=str)
disease_similarity_matrix = loadtxt(network_path + 'DSmat.txt')
mirna_similarity_matrix = loadtxt(network_path + 'RSmat.txt')
mirna_disease_associations = loadtxt(network_path + 'RDmat.txt')

DMA = mirna_disease_associations.T
DS = row_normalize(disease_similarity_matrix, True)
MS = row_normalize(mirna_similarity_matrix, True)

[num_disease, num_mirna] = DMA.shape
dim_disease = int(opts.d)
dim_mirna = int(opts.d)
dim_pred = int(opts.k)
dim_pass = int(opts.d)

# 定义结束

In [4]:
class Model(object):
    def __init__(self):
        self._build_model()

    def _build_model(self):
        self.disease_disease = tf.placeholder(tf.float32, [num_disease, num_disease])
        self.disease_disease_normalize = tf.placeholder(tf.float32, [num_disease, num_disease])
        self.mirna_mirna = tf.placeholder(tf.float32, [num_mirna, num_mirna])
        self.mirna_mirna_normalize = tf.placeholder(tf.float32, [num_mirna, num_mirna])
        self.mirna_disease = tf.placeholder(tf.float32, [num_mirna, num_disease])
        self.mirna_disease_normalize = tf.placeholder(tf.float32, [num_mirna, num_disease])
        self.disease_mirna = tf.placeholder(tf.float32, [num_disease, num_mirna])
        self.disease_mirna_normalize = tf.placeholder(tf.float32, [num_disease, num_mirna])
        self.disease_mirna_mask = tf.placeholder(tf.float32, [num_disease + num_mirna, num_disease + num_mirna])

        self.A = tf.concat([tf.concat([self.disease_disease, self.disease_mirna], 1),
                            tf.concat([self.mirna_disease, self.mirna_mirna], 1)], 0)

        self.disease_embedding = weight_variable([num_disease, dim_disease])
        self.mirna_embedding = weight_variable([num_mirna, dim_mirna])
        tf.add_to_collection('l2_reg', tf.contrib.layers.l2_regularizer(1.0)(self.disease_embedding))
        tf.add_to_collection('l2_reg', tf.contrib.layers.l2_regularizer(1.0)(self.mirna_embedding))

        W0 = weight_variable([dim_pass + dim_disease, dim_disease])
        b0 = bias_variable([dim_disease])
        tf.add_to_collection('l2_reg', tf.contrib.layers.l2_regularizer(1.0)(W0))

        disease_vector0 = tf.nn.l2_normalize(relu(tf.matmul(
            tf.concat([tf.matmul(self.disease_mirna_normalize,
                                 a_layer(self.mirna_embedding, dim_pass)) +
                       tf.matmul(self.disease_disease_normalize,
                                 a_layer(self.disease_embedding, dim_pass)),
                       self.disease_embedding], axis=1), W0) + b0), dim=1)

        mirna_vector0 = tf.nn.l2_normalize(relu(tf.matmul(
            tf.concat([tf.matmul(self.mirna_mirna_normalize,
                                 a_layer(self.mirna_embedding, dim_pass)) +
                       tf.matmul(self.mirna_disease_normalize,
                                 a_layer(self.disease_embedding, dim_pass)),
                       self.mirna_embedding], axis=1), W0) + b0), dim=1)

        self.disease_representation = disease_vector0
        self.mirna_representation = mirna_vector0
        self.features_matrix = tf.concat([self.disease_representation, self.mirna_representation], 0)
        self.A_reconstruct = bi_layer(self.features_matrix, self.features_matrix, sym=True, dim_pred=dim_pred)

        tmp = tf.multiply(self.disease_mirna_mask, (self.A_reconstruct - self.A))
        self.A_reconstruct_loss = tf.reduce_sum(tf.multiply(tmp, tmp))
        self.l2_loss = tf.add_n(tf.get_collection("l2_reg"))
        self.loss = self.A_reconstruct_loss + self.l2_loss


graph = tf.get_default_graph()
with graph.as_default():
    model = Model()
    learning_rate = tf.placeholder(tf.float32, [])
    total_loss = model.loss

    optimize = tf.train.AdamOptimizer(learning_rate)
    gradients, variables = zip(*optimize.compute_gradients(total_loss))
    gradients, _ = tf.clip_by_global_norm(gradients, int(opts.n))
    optimizer = optimize.apply_gradients(zip(gradients, variables))

    DR_pred = model.A_reconstruct[:num_disease, num_disease:]
    RD_pred = model.A_reconstruct[num_disease:, :num_disease]
    eval_pred = (DR_pred + tf.transpose(RD_pred, perm=[1,0])) / 2.0

def train_and_evaluate(DMAtrain, DMAtest, DMAcandidate, graph, verbose=True, num_steps = 2000):
    mask = np.zeros((num_disease,num_mirna))
    disease_mirna = np.zeros((num_disease,num_mirna))
    for ele in DMAtrain:
        disease_mirna[ele[0],ele[1]] = ele[2]
        mask[ele[0],ele[1]] = 1
    mirna_disease = disease_mirna.T
    disease_mirna_normalize = row_normalize(disease_mirna,False)
    mirna_disease_normalize = row_normalize(mirna_disease,False)
    mask = np.concatenate([np.concatenate([np.ones((num_disease, num_disease)), mask], axis=1),
                           np.concatenate([mask.T, np.ones((num_mirna, num_mirna))], axis=1)], axis=0)


    lr = 0.0005
    scores = []
    labels = []
    min_loss = float('inf')
    with tf.Session(graph=graph) as sess:
        tf.initialize_all_variables().run()
        for i in range(num_steps):
            _, tloss, results = sess.run([optimizer,total_loss,eval_pred],
                                        feed_dict={model.disease_mirna:disease_mirna, model.disease_mirna_normalize:disease_mirna_normalize,
                                        model.mirna_disease:mirna_disease, model.mirna_disease_normalize:mirna_disease_normalize,
                                        model.disease_disease:disease_similarity_matrix, model.mirna_mirna:mirna_similarity_matrix,
                                        model.disease_disease_normalize:DS, model.mirna_mirna_normalize:MS,
                                        model.disease_mirna_mask:mask, learning_rate: lr})
            #every 20 steps of gradient descent, evaluate the performance, other choices of this number are possible
            if i % 20 == 0 and verbose == True:
                print('step',i,'total loss',tloss)

                if tloss <= min_loss:
                    min_loss = tloss
                    best_results = results


        for ele in DMAtest:
            scores.append(best_results[ele[0], ele[1]])
            labels.append(ele[2])
        for ele in DMAcandidate:
            scores.append(best_results[ele[0], ele[1]])
            labels.append(ele[2])

    return scores, labels

def divide_known_unknown_associations(A, exception=None, special=None):
    known = []
    unknown = []
    if special != None:
        for j in range(A.shape[1]):
            if A[special][j] == 1:
                known.append([special,j,1])
            else:
                unknown.append([special,j,0])
    else:
        for i in range(A.shape[0]):
            if i == exception: pass
            for j in range(A.shape[1]):
                if A[i][j] == 1:
                    known.append([i,j,1])
                else:
                    unknown.append([i,j,0])

    return np.array(known), np.array(unknown)

def plot_roc_curve(fpr, tpr, auc):
    plt.figure(1)
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    plt.plot(fpr, tpr, color='b', label=r'Mean ROC (AUC = %0.6f)' % auc, lw=2, alpha=.8)
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc='best')
    plt.show()

def plot_pr_curve(recall, precision, aupr):
    plt.figure(2)
    plt.plot([0, 1], [1, 0], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    plt.plot(recall, precision, color='b', label=r'Mean PR (AUPR = %0.6f)' % aupr, lw=2, alpha=.8)
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='best')
    plt.show()

def FCV(epochs=2000):
    known_associations, unknown_associations = divide_known_unknown_associations(DMA)

    rs = np.random.randint(0, 1000, 1)[0]
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=rs)
    kf = kf.split(known_associations[:,:-1], known_associations[:,-1])

    tpr_fold = []
    precision_fold = []
    mean_fpr = np.linspace(0, 1, 100)
    mean_recall = np.linspace(0, 1, 100)
    DMAcandidate = unknown_associations
    for train_idx, test_idx in kf:
        DMAtrain = known_associations[train_idx]
        DMAtest = known_associations[test_idx]
        """
        # used to adjust the superparameter
        DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                            graph=graph, num_steps=epochs)
        """
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest, DMAcandidate=DMAcandidate,
                                            graph=graph, num_steps=epochs)

        fpr, tpr, _ = roc_curve(labels, scores)
        interp_tpr = interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tpr_fold.append(interp_tpr)

        precision, recall, _ = precision_recall_curve(labels, scores)
        rank_idx = np.argsort(recall)
        recall = recall[rank_idx]
        precision = precision[rank_idx]
        interp_precision = interp(mean_recall, recall, precision)
        interp_precision[0] = 1.0
        precision_fold.append(interp_precision)

    mean_tpr = np.mean(tpr_fold, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    #plot_roc_curve(mean_fpr, mean_tpr, mean_auc)
    roc = pd.DataFrame({'fpr':mean_fpr,'tpr':mean_tpr})
    roc.to_csv(save_path + '5fold_roc.csv', index=False)

    mean_precision = np.mean(precision_fold, axis=0)
    mean_precision[-1] = 0.0
    mean_aupr = auc(mean_recall, mean_precision)
    #plot_pr_curve(mean_recall, mean_precision, mean_aupr)
    pr = pd.DataFrame({'recall': mean_recall, 'precision': mean_precision})
    pr.to_csv(save_path + '5fold_pr.csv', index=False)

    print('5-fold auc %0.6f', mean_auc, ', 5-fold aupr %0.6f', mean_aupr)


def LODOCV(epochs=2000):
    tpr_fold = []
    precision_fold = []
    mean_fpr = np.linspace(0, 1, 100)
    mean_recall = np.linspace(0, 1, 100)
    for i in range(num_disease):
        print('current cv: %d, total cv: %d' % (i+1, num_disease))

        DMAtrain, _ = divide_known_unknown_associations(DMA, exception=i)
        DMAtest, DMAcandidate = divide_known_unknown_associations(DMA, special=i)
        """
        # used to adjust the superparameter
        rs = np.random.randint(0, 1000, 1)[0]
        DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                            graph=graph, num_steps=epochs)
        """
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest, DMAcandidate=DMAcandidate,
                                            graph=graph, num_steps=epochs)

        fpr, tpr, _ = roc_curve(labels, scores)
        interp_tpr = interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tpr_fold.append(interp_tpr)

        precision, recall, _ = precision_recall_curve(labels, scores)
        rank_idx = np.argsort(recall)
        recall = recall[rank_idx]
        precision = precision[rank_idx]
        interp_precision = interp(mean_recall, recall, precision)
        interp_precision[0] = 1.0
        precision_fold.append(interp_precision)

    mean_tpr = np.mean(tpr_fold, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    #plot_roc_curve(mean_fpr, mean_tpr, mean_auc)
    roc = pd.DataFrame({'fpr': mean_fpr, 'tpr': mean_tpr})
    roc.to_csv(save_path + 'lodocv_roc.csv', index=False)

    mean_precision = np.mean(precision_fold, axis=0)
    mean_precision[-1] = 0.0
    mean_aupr = auc(mean_recall, mean_precision)
    #plot_pr_curve(mean_recall, mean_precision, mean_aupr)
    pr = pd.DataFrame({'recall': mean_recall, 'precision': mean_precision})
    pr.to_csv(save_path + 'lodocv_pr.csv', index=False)

    print('lodocv auc %0.6f', mean_auc, ', lodocv aupr %0.6f', mean_aupr)

def GLOOCV(epochs):
    known_associations, unknown_associations = divide_known_unknown_associations(DMA)

    tpr_fold = []
    precision_fold = []
    n = len(known_associations)
    mean_fpr = np.linspace(0, 1, 100)
    mean_recall = np.linspace(0, 1, 100)
    DMAcandidate = unknown_associations
    for i in range(n):
        print('current cv: %d, total cv: %d' % (i+1, n))

        ele = known_associations[i]
        DMAtest = [ele]
        DMAtrain = np.delete(known_associations, i, axis=0)
        """
        # used to adjust the superparameter
        rs = np.random.randint(0, 1000, 1)[0]
        DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                            graph=graph, num_steps=epochs)
        """
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest, DMAcandidate=DMAcandidate,
                                            graph=graph, num_steps=epochs)

        fpr, tpr, _ = roc_curve(labels, scores)
        interp_tpr = interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tpr_fold.append(interp_tpr)

        precision, recall, _ = precision_recall_curve(labels, scores)
        rank_idx = np.argsort(recall)
        recall = recall[rank_idx]
        precision = precision[rank_idx]
        interp_precision = interp(mean_recall, recall, precision)
        interp_precision[0] = 1.0
        precision_fold.append(interp_precision)

    mean_tpr = np.mean(tpr_fold, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    #plot_roc_curve(mean_fpr, mean_tpr, mean_auc)
    roc = pd.DataFrame({'fpr': mean_fpr, 'tpr': mean_tpr})
    roc.to_csv(save_path + 'gloocv_roc.csv', index=False)

    mean_precision = np.mean(precision_fold, axis=0)
    mean_precision[-1] = 0.0
    mean_aupr = auc(mean_recall, mean_precision)
    #plot_pr_curve(mean_recall, mean_precision, mean_aupr)
    pr = pd.DataFrame({'recall': mean_recall, 'precision': mean_precision})
    pr.to_csv(save_path + 'gloocv_pr.csv', index=False)

    print('global loocv auc %0.6f' % mean_auc, ', global loocv aupr %0.6f' % mean_aupr)

def LLOOCV(epochs):
    known_associations, unknown_associations = divide_known_unknown_associations(DMA)

    tpr_fold = []
    precision_fold = []
    n = len(known_associations)
    mean_fpr = np.linspace(0, 1, 100)
    mean_recall = np.linspace(0, 1, 100)
    for i in range(n):
        print('current cv: %d, total cv: %d' % (i+1, n))

        ele = known_associations[i]
        DMAtest = [ele]
        DMAtrain = np.delete(known_associations, i, axis=0)
        _, DMAcandidate = divide_known_unknown_associations(DMA, special=ele[0])
        """
        # used to adjust the superparameter
        rs = np.random.randint(0, 1000, 1)[0]
        DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                            graph=graph, num_steps=epochs)
        """
        scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest, DMAcandidate=DMAcandidate,
                                            graph=graph, num_steps=epochs)

        fpr, tpr, _ = roc_curve(labels, scores)
        interp_tpr = interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tpr_fold.append(interp_tpr)

        precision, recall, _ = precision_recall_curve(labels, scores)
        rank_idx = np.argsort(recall)
        recall = recall[rank_idx]
        precision = precision[rank_idx]
        interp_precision = interp(mean_recall, recall, precision)
        interp_precision[0] = 1.0
        precision_fold.append(interp_precision)

    mean_tpr = np.mean(tpr_fold, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    #plot_roc_curve(mean_fpr, mean_tpr, mean_auc)
    roc = pd.DataFrame({'fpr': mean_fpr, 'tpr': mean_tpr})
    roc.to_csv(save_path + 'lloocv_roc.csv', index=False)

    mean_precision = np.mean(precision_fold, axis=0)
    mean_precision[-1] = 0.0
    mean_aupr = auc(mean_recall, mean_precision)
    #plot_pr_curve(mean_recall, mean_precision, mean_aupr)
    pr = pd.DataFrame({'recall': mean_recall, 'precision': mean_precision})
    pr.to_csv(save_path + 'lloocv_pr.csv', index=False)

    print('local loocv auc %0.6f' % mean_auc, ', local loocv aupr %0.6f' % mean_aupr)

def case_study1(epochs=2000):
    DMAtest = []
    DMAtrain, _ = divide_known_unknown_associations(DMA)
    _, DMAcandidate = divide_known_unknown_associations(DMA, special=288)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest, DMAcandidate=DMAcandidate,
                                        graph=graph, num_steps=epochs)

    index = []
    for ele in DMAcandidate:
        index.append(ele[1])

    index = np.array(index)
    scores = np.array(scores)
    rank_idx = np.argsort(scores)
    index = index[rank_idx]
    index = index[-30:]

    for i in index:
        print(mirna_name[i])


def case_study2(epochs=2000):
    DMAtrain, _ = divide_known_unknown_associations(DMA, exception=271)
    DMAtest, DMAcandidate = divide_known_unknown_associations(DMA, special=271)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest, DMAcandidate=DMAcandidate,
                                        graph=graph, num_steps=epochs)

    index = []
    for ele in DMAtest:
        index.append(ele[1])
    for ele in DMAcandidate:
        index.append(ele[1])

    index = np.array(index)
    scores = np.array(scores)
    rank_idx = np.argsort(scores)
    index = index[rank_idx]
    index = index[-50:]

    for i in index:
        print(mirna_name[i])


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# 思考

In [5]:
def performances(y_true, y_pred, y_prob):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if tp+fn == 0:
        recall = 0
    else:
        recall = tp / (tp+fn)
    
    if tp+fp == 0:
        precision = 0
    else:
        precision = tp / (tp+fp)
    
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr)

def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred.reshape(-1)]

In [6]:
def train_and_evaluate(DMAtrain, DMAtest, graph, verbose=True, num_steps = 2000):
    mask = np.zeros((num_disease,num_mirna))
    disease_mirna = np.zeros((num_disease,num_mirna))
    for ele in DMAtrain:
        disease_mirna[ele[0],ele[1]] = ele[2]
        mask[ele[0],ele[1]] = 1
    mirna_disease = disease_mirna.T
    disease_mirna_normalize = row_normalize(disease_mirna,False)
    mirna_disease_normalize = row_normalize(mirna_disease,False)
    mask = np.concatenate([np.concatenate([np.ones((num_disease, num_disease)), mask], axis=1),
                           np.concatenate([mask.T, np.ones((num_mirna, num_mirna))], axis=1)], axis=0)


    lr = 0.0005
    scores = []
    labels = []
    min_loss = float('inf')
    with tf.Session(graph=graph) as sess:
        tf.initialize_all_variables().run()
        for i in trange(num_steps):
            _, tloss, results = sess.run([optimizer,total_loss,eval_pred],
                                        feed_dict={model.disease_mirna:disease_mirna, model.disease_mirna_normalize:disease_mirna_normalize,
                                        model.mirna_disease:mirna_disease, model.mirna_disease_normalize:mirna_disease_normalize,
                                        model.disease_disease:disease_similarity_matrix, model.mirna_mirna:mirna_similarity_matrix,
                                        model.disease_disease_normalize:DS, model.mirna_mirna_normalize:MS,
                                        model.disease_mirna_mask:mask, learning_rate: lr})
            #every 20 steps of gradient descent, evaluate the performance, other choices of this number are possible
            if i % 20 == 0 and verbose == True:
#                 print('step',i,'total loss',tloss)

                if tloss <= min_loss:
                    min_loss = tloss
                    best_results = results


        for ele in DMAtest:
            scores.append(best_results[ele[0], ele[1]])
            labels.append(ele[2])
        

    return scores, labels

def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred]

# Tp, balance

In [7]:
known_associations, unknown_associations = divide_known_unknown_associations(DMA)
dtp = pd.DataFrame(np.vstack([known_associations, unknown_associations]), columns = ['disease', 'miRNA', 'label'])
dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num = obtain_data(dtp, isbalance = True)
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(dtp)

5it [00:00, 202.05it/s]

# miRNA = 577 | Disease = 336
# Test: miRNA = 115 | Disease = 67
-------Fold  0
# Pairs: Train = 10100 | Test = 2526
-------Fold  1
# Pairs: Train = 10101 | Test = 2525
-------Fold  2
# Pairs: Train = 10101 | Test = 2525
-------Fold  3
# Pairs: Train = 10101 | Test = 2525
-------Fold  4
# Pairs: Train = 10101 | Test = 2525


In [8]:
epochs = 2000

tpr_fold = []
precision_fold = []
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
# DMAcandidate = unknown_associations
for train_idx, test_idx in zip(train_index_all, test_index_all):
    DMAtrain = np.array(dtp.iloc[train_idx])
    DMAtest = np.array(dtp.iloc[test_idx])

    """
    # used to adjust the superparameter
    DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                        graph=graph, num_steps=epochs)
    """
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest,
                                        graph=graph, num_steps=epochs)
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


100%|██████████| 2000/2000 [05:05<00:00,  6.54it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1273, fp = 3, fn = 719, tp = 531
y_pred: 0 = 1992 | 1 = 534
y_true: 0 = 1276 | 1 = 1250
acc=0.7142|precision=0.9944|recall=0.4248|f1=0.5953|auc=0.9694|aupr=0.9717


100%|██████████| 2000/2000 [05:17<00:00,  6.29it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1275, fp = 5, fn = 708, tp = 537
y_pred: 0 = 1983 | 1 = 542
y_true: 0 = 1280 | 1 = 1245
acc=0.7176|precision=0.9908|recall=0.4313|f1=0.6010|auc=0.9688|aupr=0.9688


...

tn = 1231, fp = 5, fn = 751, tp = 538
y_pred: 0 = 1982 | 1 = 543
y_true: 0 = 1236 | 1 = 1289
acc=0.7006|precision=0.9908|recall=0.4174|f1=0.5873|auc=0.9633|aupr=0.9666


100%|██████████| 2000/2000 [03:28<00:00,  9.61it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1249, fp = 4, fn = 710, tp = 562
y_pred: 0 = 1959 | 1 = 566
y_true: 0 = 1253 | 1 = 1272
acc=0.7172|precision=0.9929|recall=0.4418|f1=0.6115|auc=0.9655|aupr=0.9682


100%|██████████| 2000/2000 [03:21<00:00,  9.94it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1263, fp = 5, fn = 790, tp = 467
y_pred: 0 = 2053 | 1 = 472
y_true: 0 = 1268 | 1 = 1257
acc=0.6851|precision=0.9894|recall=0.3715|f1=0.5402|auc=0.9601|aupr=0.9610


# Tm, balance

In [10]:
known_associations, unknown_associations = divide_known_unknown_associations(DMA)
dtp = pd.DataFrame(np.vstack([known_associations, unknown_associations]), columns = ['disease', 'miRNA', 'label'])
dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num = obtain_data(dtp, isbalance = True)
item = 'miRNA'
ids = mirna_ids
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)


epochs = 2000

tpr_fold = []
precision_fold = []
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
# DMAcandidate = unknown_associations
for train_idx, test_idx in zip(train_index_all, test_index_all):
    DMAtrain = np.array(dtp.iloc[train_idx])
    DMAtest = np.array(dtp.iloc[test_idx])

    """
    # used to adjust the superparameter
    DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                        graph=graph, num_steps=epochs)
    """
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest,
                                        graph=graph, num_steps=epochs)
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

# miRNA = 577 | Disease = 336
# Test: miRNA = 115 | Disease = 67
-------Fold  0
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 9783 | Test = 2843
-------Fold  1
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 10233 | Test = 2393
-------Fold  2
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 10096 | Test = 2530
-------Fold  3
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 10296 | Test = 2330
-------Fold  4
# miRNA: Train = 460 | Test = 117
# Pairs: Train = 10096 | Test = 2530


100%|██████████| 2000/2000 [03:23<00:00,  9.84it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1228, fp = 5, fn = 1066, tp = 544
y_pred: 0 = 2294 | 1 = 549
y_true: 0 = 1233 | 1 = 1610
acc=0.6233|precision=0.9909|recall=0.3379|f1=0.5039|auc=0.9569|aupr=0.9656


100%|██████████| 2000/2000 [03:24<00:00,  9.77it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1259, fp = 4, fn = 579, tp = 551
y_pred: 0 = 1838 | 1 = 555
y_true: 0 = 1263 | 1 = 1130
acc=0.7564|precision=0.9928|recall=0.4876|f1=0.6540|auc=0.9768|aupr=0.9766


...

tn = 1253, fp = 3, fn = 778, tp = 496
y_pred: 0 = 2031 | 1 = 499
y_true: 0 = 1256 | 1 = 1274
acc=0.6913|precision=0.9940|recall=0.3893|f1=0.5595|auc=0.9639|aupr=0.9641


100%|██████████| 2000/2000 [03:18<00:00, 10.07it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1294, fp = 5, fn = 575, tp = 456
y_pred: 0 = 1869 | 1 = 461
y_true: 0 = 1299 | 1 = 1031
acc=0.7511|precision=0.9892|recall=0.4423|f1=0.6113|auc=0.9645|aupr=0.9591


100%|██████████| 2000/2000 [03:20<00:00,  9.99it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1255, fp = 7, fn = 707, tp = 561
y_pred: 0 = 1962 | 1 = 568
y_true: 0 = 1262 | 1 = 1268
acc=0.7178|precision=0.9877|recall=0.4424|f1=0.6111|auc=0.9614|aupr=0.9635


# Td, balance

In [12]:
known_associations, unknown_associations = divide_known_unknown_associations(DMA)
dtp = pd.DataFrame(np.vstack([known_associations, unknown_associations]), columns = ['disease', 'miRNA', 'label'])
dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num = obtain_data(dtp, isbalance = True)
item = 'disease'
ids = disease_ids
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)


epochs = 2000

tpr_fold = []
precision_fold = []
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
# DMAcandidate = unknown_associations
for train_idx, test_idx in zip(train_index_all, test_index_all):
    DMAtrain = np.array(dtp.iloc[train_idx])
    DMAtest = np.array(dtp.iloc[test_idx])

    """
    # used to adjust the superparameter
    DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                        graph=graph, num_steps=epochs)
    """
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest,
                                        graph=graph, num_steps=epochs)
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

# miRNA = 577 | Disease = 336
# Test: miRNA = 115 | Disease = 67
-------Fold  0
# disease: Train = 269 | Test = 67
# Pairs: Train = 9842 | Test = 2784
-------Fold  1
# disease: Train = 269 | Test = 67
# Pairs: Train = 10202 | Test = 2424
-------Fold  2
# disease: Train = 269 | Test = 67
# Pairs: Train = 10245 | Test = 2381
-------Fold  3
# disease: Train = 269 | Test = 67
# Pairs: Train = 9814 | Test = 2812
-------Fold  4
# disease: Train = 268 | Test = 68
# Pairs: Train = 10401 | Test = 2225


100%|██████████| 2000/2000 [03:34<00:00,  9.33it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1240, fp = 0, fn = 1521, tp = 23
y_pred: 0 = 2761 | 1 = 23
y_true: 0 = 1240 | 1 = 1544
acc=0.4537|precision=1.0000|recall=0.0149|f1=0.0294|auc=0.8549|aupr=0.8825


100%|██████████| 2000/2000 [03:52<00:00,  8.59it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1293, fp = 1, fn = 1113, tp = 17
y_pred: 0 = 2406 | 1 = 18
y_true: 0 = 1294 | 1 = 1130
acc=0.5404|precision=0.9444|recall=0.0150|f1=0.0296|auc=0.8639|aupr=0.8560


...

tn = 1277, fp = 1, fn = 1085, tp = 18
y_pred: 0 = 2362 | 1 = 19
y_true: 0 = 1278 | 1 = 1103
acc=0.5439|precision=0.9474|recall=0.0163|f1=0.0321|auc=0.8643|aupr=0.8478


100%|██████████| 2000/2000 [03:25<00:00,  9.72it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1222, fp = 0, fn = 1574, tp = 16
y_pred: 0 = 2796 | 1 = 16
y_true: 0 = 1222 | 1 = 1590
acc=0.4403|precision=1.0000|recall=0.0101|f1=0.0199|auc=0.8764|aupr=0.8985


100%|██████████| 2000/2000 [04:11<00:00,  7.96it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 1279, fp = 0, fn = 918, tp = 28
y_pred: 0 = 2197 | 1 = 28
y_true: 0 = 1279 | 1 = 946
acc=0.5874|precision=1.0000|recall=0.0296|f1=0.0575|auc=0.8522|aupr=0.8135


# Tp, unbalanced

In [13]:
known_associations, unknown_associations = divide_known_unknown_associations(DMA)
dtp = pd.DataFrame(np.vstack([known_associations, unknown_associations]), columns = ['disease', 'miRNA', 'label'])
dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num = obtain_data(dtp, isbalance = False)
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(dtp)

epochs = 2000

tpr_fold = []
precision_fold = []
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
# DMAcandidate = unknown_associations
for train_idx, test_idx in zip(train_index_all, test_index_all):
    DMAtrain = np.array(dtp.iloc[train_idx])
    DMAtest = np.array(dtp.iloc[test_idx])

    """
    # used to adjust the superparameter
    DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                        graph=graph, num_steps=epochs)
    """
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest,
                                        graph=graph, num_steps=epochs)
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

1it [00:00,  7.73it/s]

# miRNA = 577 | Disease = 336
# Test: miRNA = 115 | Disease = 67
-------Fold  0
# Pairs: Train = 155097 | Test = 38775
-------Fold  1
# Pairs: Train = 155097 | Test = 38775
-------Fold  2


5it [00:00, 16.10it/s]


# Pairs: Train = 155098 | Test = 38774
-------Fold  3
# Pairs: Train = 155098 | Test = 38774
-------Fold  4
# Pairs: Train = 155098 | Test = 38774


100%|██████████| 2000/2000 [03:54<00:00,  8.53it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


...

tn = 37426, fp = 37, fn = 933, tp = 378
y_pred: 0 = 38359 | 1 = 415
y_true: 0 = 37463 | 1 = 1311
acc=0.9750|precision=0.9108|recall=0.2883|f1=0.4380|auc=0.9622|aupr=0.7141


100%|██████████| 2000/2000 [03:18<00:00, 10.09it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37501, fp = 31, fn = 915, tp = 327
y_pred: 0 = 38416 | 1 = 358
y_true: 0 = 37532 | 1 = 1242
acc=0.9756|precision=0.9134|recall=0.2633|f1=0.4088|auc=0.9691|aupr=0.7210


100%|██████████| 2000/2000 [03:10<00:00, 10.52it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37507, fp = 17, fn = 991, tp = 259
y_pred: 0 = 38498 | 1 = 276
y_true: 0 = 37524 | 1 = 1250
acc=0.9740|precision=0.9384|recall=0.2072|f1=0.3394|auc=0.9652|aupr=0.7367


# Tm, unbalanced

In [14]:
known_associations, unknown_associations = divide_known_unknown_associations(DMA)
dtp = pd.DataFrame(np.vstack([known_associations, unknown_associations]), columns = ['disease', 'miRNA', 'label'])
dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num = obtain_data(dtp, isbalance = False)
item = 'miRNA'
ids = mirna_ids
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)


epochs = 2000

tpr_fold = []
precision_fold = []
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
# DMAcandidate = unknown_associations
for train_idx, test_idx in zip(train_index_all, test_index_all):
    DMAtrain = np.array(dtp.iloc[train_idx])
    DMAtest = np.array(dtp.iloc[test_idx])

    """
    # used to adjust the superparameter
    DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                        graph=graph, num_steps=epochs)
    """
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest,
                                        graph=graph, num_steps=epochs)
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

# miRNA = 577 | Disease = 336
# Test: miRNA = 115 | Disease = 67
-------Fold  0
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 155232 | Test = 38640
-------Fold  1
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 155232 | Test = 38640
-------Fold  2
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 155232 | Test = 38640
-------Fold  3
# miRNA: Train = 462 | Test = 115
# Pairs: Train = 155232 | Test = 38640
-------Fold  4
# miRNA: Train = 460 | Test = 117
# Pairs: Train = 154560 | Test = 39312


100%|██████████| 2000/2000 [03:08<00:00, 10.60it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37122, fp = 29, fn = 1148, tp = 341
y_pred: 0 = 38270 | 1 = 370
y_true: 0 = 37151 | 1 = 1489
acc=0.9695|precision=0.9216|recall=0.2290|f1=0.3669|auc=0.9541|aupr=0.6952


100%|██████████| 2000/2000 [03:07<00:00, 10.67it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37155, fp = 35, fn = 1089, tp = 361
y_pred: 0 = 38244 | 1 = 396
y_true: 0 = 37190 | 1 = 1450
acc=0.9709|precision=0.9116|recall=0.2490|f1=0.3911|auc=0.9503|aupr=0.6634


...

tn = 37585, fp = 8, fn = 733, tp = 314
y_pred: 0 = 38318 | 1 = 322
y_true: 0 = 37593 | 1 = 1047
acc=0.9808|precision=0.9752|recall=0.2999|f1=0.4587|auc=0.9701|aupr=0.7619


100%|██████████| 2000/2000 [03:19<00:00, 10.05it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37302, fp = 41, fn = 978, tp = 319
y_pred: 0 = 38280 | 1 = 360
y_true: 0 = 37343 | 1 = 1297
acc=0.9736|precision=0.8861|recall=0.2460|f1=0.3850|auc=0.9606|aupr=0.6753


100%|██████████| 2000/2000 [03:14<00:00, 10.27it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 38234, fp = 48, fn = 673, tp = 357
y_pred: 0 = 38907 | 1 = 405
y_true: 0 = 38282 | 1 = 1030
acc=0.9817|precision=0.8815|recall=0.3466|f1=0.4976|auc=0.9736|aupr=0.7059


# Td, unbalanced

In [15]:
known_associations, unknown_associations = divide_known_unknown_associations(DMA)
dtp = pd.DataFrame(np.vstack([known_associations, unknown_associations]), columns = ['disease', 'miRNA', 'label'])
dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num = obtain_data(dtp, isbalance = False)
item = 'disease'
ids = disease_ids
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)


epochs = 2000

tpr_fold = []
precision_fold = []
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
# DMAcandidate = unknown_associations
for train_idx, test_idx in zip(train_index_all, test_index_all):
    DMAtrain = np.array(dtp.iloc[train_idx])
    DMAtest = np.array(dtp.iloc[test_idx])

    """
    # used to adjust the superparameter
    DMAtrain, DMAvalid = train_test_split(DMAtrain, test_size=0.05, random_state=rs)
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAvalid, DMAcandidate=DMAcandidate, 
                                        graph=graph, num_steps=epochs)
    """
    scores, labels = train_and_evaluate(DMAtrain=DMAtrain, DMAtest=DMAtest,
                                        graph=graph, num_steps=epochs)
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

# miRNA = 577 | Disease = 336
# Test: miRNA = 115 | Disease = 67
-------Fold  0
# disease: Train = 269 | Test = 67
# Pairs: Train = 155213 | Test = 38659
-------Fold  1
# disease: Train = 269 | Test = 67
# Pairs: Train = 155213 | Test = 38659
-------Fold  2
# disease: Train = 269 | Test = 67
# Pairs: Train = 155213 | Test = 38659
-------Fold  3
# disease: Train = 269 | Test = 67
# Pairs: Train = 155213 | Test = 38659
-------Fold  4
# disease: Train = 268 | Test = 68
# Pairs: Train = 154636 | Test = 39236


100%|██████████| 2000/2000 [03:35<00:00,  9.30it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37292, fp = 0, fn = 1366, tp = 1
y_pred: 0 = 38658 | 1 = 1
y_true: 0 = 37292 | 1 = 1367
acc=0.9647|precision=1.0000|recall=0.0007|f1=0.0015|auc=0.8214|aupr=0.2714


100%|██████████| 2000/2000 [03:24<00:00,  9.76it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37183, fp = 0, fn = 1472, tp = 4
y_pred: 0 = 38655 | 1 = 4
y_true: 0 = 37183 | 1 = 1476
acc=0.9619|precision=1.0000|recall=0.0027|f1=0.0054|auc=0.8503|aupr=0.2518


...

tn = 37662, fp = 0, fn = 997, tp = 0
y_pred: 0 = 38659 | 1 = 0
y_true: 0 = 37662 | 1 = 997
acc=0.9742|precision=0.0000|recall=0.0000|f1=0.0000|auc=0.8674|aupr=0.3103


100%|██████████| 2000/2000 [03:58<00:00,  8.40it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 37384, fp = 0, fn = 1271, tp = 4
y_pred: 0 = 38655 | 1 = 4
y_true: 0 = 37384 | 1 = 1275
acc=0.9671|precision=1.0000|recall=0.0031|f1=0.0063|auc=0.8253|aupr=0.2759


100%|██████████| 2000/2000 [03:45<00:00,  8.87it/s]
/home/chujunyi/anaconda3/envs/tensorflow_1.8/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


tn = 38038, fp = 0, fn = 1197, tp = 1
y_pred: 0 = 39235 | 1 = 1
y_true: 0 = 38038 | 1 = 1198
acc=0.9695|precision=1.0000|recall=0.0008|f1=0.0017|auc=0.8093|aupr=0.2268
